In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 11 #

## Lists

In [ ]:
# Recall that a list is a sequence of values, and the values can be different types
mylist = ...
mylist

In [ ]:
# We can access individual items in a list with the "bracket operator" (indexing operator)
...

In [ ]:
...

In [ ]:
# If we use an invalid index, we get an IndexError
...

In [ ]:
# The largest valid index is one less than the length of the list
max_valid = len(mylist) - 1
max_valid

## Rows from lists

In [ ]:
# We already know how to make a new table by providing arrays for the columns
# Use the `with_columns` method:
Table().with_columns('Numbers', make_array(1, 2, 3))

In [ ]:
# If we want, we can use a list instead of an array
Table().with_columns('Numbers', [1, 2, 3])

In [ ]:
# We can also make a new table by providing lists for the ROWS
# We start by making a rowless table with three column headings
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

In [ ]:
# Then use the `with_rows` method, passing in a "list of lists" for the row values
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

In [ ]:
# Here's the list of lists which describes the sequence of rows
list_of_rows = [
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
]
list_of_rows

**Question**: Why can't an `array` hold the row information for a row of the drinks table? Why use a `list` instead?

**Back to Slides...**

## Grouping by Two Categorical Variables (aka Cross-Classification)

In [ ]:
# These are the Data 8 "Welcome Survey" data from Spring 2022
survey = Table.read_table('welcome_survey_sp22.csv')
survey.show(3)

We're going to group the survey data by 'Handedness' and 'Sleep position' to investigate a possible association between the two. It's a good idea to first group by each column individually, to familiarize ourselves with the distributions of those variables.

In [ ]:
survey.group('Handedness')

In [ ]:
survey.group('Sleep position')

There are 3 unique values for 'Handedness' and 4 for 'Sleep position'. 

  - How many rows might we get when we group on both variables simultaneously?
  - Is there an association between handedness (right/left) and preferred side for sleeping (right/left)?

In [ ]:
# Notice the syntax: The two column labels are put into a list, and that list
# is the first argument to `group`
survey.group(['Handedness', 'Sleep position']).show()

In [ ]:
# to answer the second question, focus on just the 4 relevant rows
survey.group(['Handedness', 'Sleep position']).take(5, 6, 9, 10)

Leftie side-sleepers are almost evenly split between preferring to sleep on one side or the other. Rightie side-sleepers seem to be somewhat more inclined to sleep on their right.

We can also include a second argument (an aggregating function). Check this out:

In [ ]:
survey.group(['Handedness', 'Sleep position'], np.average).sort(8, descending=True).show()

Hmm. Looks like Data 8 students who are ambidextrous and sleep on their backs tend to have a lot of piercings! At least, it was true in Spring 2022, according to the survey responses.

**Back to Slides...**

## Compare group and pivot

A pivot table is similar to grouping with two categorical variables.

In [ ]:
# Using group
survey.group(['Handedness', 'Sleep position']).show()

In [ ]:
# Using pivot
survey.pivot('Handedness', 'Sleep position').show()

These tables show the exact same information but with different formats.

To show aggregated values of some third column instead of counts, we have to provide the label for the third variable and the name of the aggregating function.

In [ ]:
print("Average Sleep Hours, by Handedness and Sleep Position")
survey.pivot('Handedness', 'Sleep position', 'Hours of sleep', np.average).show()

**Back to Slides...**

## Discussion Questions ##

In [ ]:
# From the CORGIS Dataset Project
# By Austin Cory Bart acbart@vt.edu
# Version 2.0.0, created 3/22/2016
# https://corgis-edu.github.io/corgis/csv/skyscrapers/
sky = Table.read_table('skyscrapers.csv')
sky.show(5)

In [ ]:
# Question 1. For each city, how tall is the tallest building for each material?

# Let's drop the 'completed' column and sort by city
sky1 = sky.drop('completed').sort('city')
sky1.show(10)

In [ ]:
# group by city and material
sky1.group(['city', 'material'])

In [ ]:
# Find the name and height of the tallest concrete skyscraper in Atlanta
heights_table = sky1.where('city', 'Atlanta').where('material', 'concrete')
heights_array = heights_table.column('height')
max_height = np.max(heights_array)
heights_table.where('height', max_height)

**SunTrust Plaza is the winner, with a height of 264.25 meters.**

In [ ]:
# To solve the overall question, use group() to cross-classify the data using city and material. 
# Provide max as the aggregating function, to learn the max heights.

max_table = sky1.group(['city', 'material'], max)
max_table

**Carefully inspect the result.**

  - Is 264.25 the correct height for the tallest concrete skyscraper in Atlanta?
  - Is Westin Peachtree Plaza the correct name?
  - How does `max` operate on an array of strings?

In [ ]:
# "name max" is not what we want
max_table = max_table.drop('name max').relabeled(2, 'maximum height')
max_table.show(5)

So, question #1 is solved. For each city and each material, max_table shows the height of the tallest skyscraper.

Could we have used a pivot table instead? 

In [ ]:
sky1.show(3)  # remember what information is held by sky1

In [ ]:
# Cross-classify by putting the different building materials into the column labels 
# and using the city names as the row labels, using pivot:
sky1.pivot(...)

Instead of showing the counts in our pivot table, we want to show the max heights. We can do this with `np.max` as the aggregating function and "height" as the value to be aggregated. The aggregating function is the fourth argument to `pivot`:

In [ ]:
sky_p = sky1.pivot(....)
sky_p

Which solution was simpler; the one using `group`, or the one using `pivot`?

**Question 2**: Instead of showing the max height, show the name of the building with the max height (for that combination of city and material). 

Here's a solution using `group` and `apply`. We start with a custom function which we will then `apply` to `max_table`.

In [ ]:
# Question: Define a function which takes a row index k (for max_table)
# and finds the name of the corresponding building in the sky1 table
def find_name(k):
    # c is the city
    c = max_table.column('city').item(k)
    # m is the material
    m = max_table.column('material').item(k)
    # h is the height
    h = max_table.column('maximum height').item(k)
    # matches should have just one row
    matches = sky1.where('city', c).where('material', m).where('height', h)
    
    return matches.column('name').item(0)

# which building is max_table's fourth row? (Austin, concrete, 208.15)
find_name(3)

In [ ]:
# Recall what information is in `max_table`
max_table.show(3)

In [ ]:
# Add a column called 'row index'; to use `apply` with our function `find_name`, we need
# the relevant row index
max_table = max_table.with_column(...)
max_table

In [ ]:
# Apply `find_name` to compute (and remember) the array of building names for the rows of `max_table`
names = max_table.apply(...)
names

In [ ]:
# Drop the row index column, and add the name column to max_table
max_table = max_table.drop('row index').with_column('name', names)

Outstanding! Before moving on to the next challenge, let's check out the rows of `max_table` for some nearby cities.

In [ ]:
max_table.where('city', 'Cincinnati')

In [ ]:
max_table.where('city', 'Indianapolis')

Here's an image of the Chase Tower: ![Chase Tower](https://www.connectcre.com/wp-content/uploads/2021/02/mw_111monumentcircle_indianapolis.jpg)

In [ ]:
# Question 3. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?

# Hint: Use the pivot table from the previous question to compute
# the differences and add that new column to the pivot table

diff = sky_p.column('steel') - sky_p.column('concrete')
sky_p_with_difference = sky_p.with_column('steel vs. concrete difference', diff)
sky_p_with_difference

In [ ]:
# Write code to count the rows in the previous table where the difference is negative
sky_p_with_difference.where(4, are.below(0)).num_rows

### Another Challenge

In [ ]:
# Question 4. For each material and each city (cross-classification), find the name of the oldest 
# skyscraper. Show the results in a table.

# Let's start by reviewing the sky table:
sky.show(5)

In [ ]:
# Hint: You can use sort to find the name of the oldest skyscraper in Chicago
old_chicago = ...
print("Oldest in Chicago:", old_chicago)

In [ ]:
# Define a function, first, which accepts an array of values and returns
# the item at index 0. We'll use it with "pivot", below...

def first(my_array):
    '''Takes a non-empty array of values and returns the first item.'''
    return my_array.item(0)

# a little test
nums = make_array(4, 9, 3, 1, 5)
first(nums)

In [ ]:
# Sort the rows in ascending order based on the year in which the building was completed
# so that when we take the rows for a specific city-material combination, the "first" name
# will tell us the oldest building
sky.sort('completed')

In [ ]:
# Now call on pivot with four arguments; have it show the first name in each group
sky.sort('completed').pivot('material', 'city', 'name', first)

That's an impressively short block of code, for what seemed like a real challenge. Read up on the `pivot` method in the `datascience` documentation and be sure you understand how that last line of code did the trick.

If you're feeling a little shaky concerning all these table, manipulations, that's OK! You have a project 1 to work on, and it will give you the practice you need to solidify these skills. 

Just be sure to spend some time **each day** working on your codes for this class. You cannot build a skill without regular practice.

And please seek out the help you need when you are stuck! You have classmates, tutors, and a professor to assist you.

**Back to Slides...**

## Joins ##

In [ ]:
# From the beginning of lecture...
drinks

In [ ]:
# Here's the information about discounts
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

In [ ]:
# Combine the tables using `join`
combined = drinks.join('Cafe', discounts, 'Location')
combined

In [ ]:
# Add a column which shows the discounted price
discount_proportion = combined.column('Coupon % off') / 100
discount_dollars = combined.column('Price') * discount_proportion
combined.with_column(
    'Discounted Price', 
    np.round(combined.column('Price') - discount_dollars, 2)
)

In [ ]:
# What happens when we join the drinks table with itself on the 'Cafe' column?
drinks.show()
drinks.join('Cafe', drinks, 'Cafe')

For each cafe, we see all the options for ordering a first drink and a second drink.